# Working on meat_0 JSON File

## 1. Parsing meat file

### Initialising and processing data

In [1]:
# downloading libraries

!pip install beautifulsoup4

In [2]:
# importing libraries

import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [3]:
# loading in meat json

with open('meat_0.json', encoding="utf-8") as f:
    data = json.load(f)

In [4]:
# making a Pandas DF with only IDs and rawHTMLs

id_list = []
html_list = []
for row in data:
    entry_id = row["_id"]
    raw_html = row["_source"]["rawHtml"]
    
    id_list.append(entry_id)
    html_list.append(raw_html)

In [5]:
df = pd.DataFrame(list(zip(id_list, html_list)), columns = ['entry_id', 'raw_html'])

In [6]:
df

,entry_id,raw_html
0,EVNGX3gBPctInV7JLxjC,"<html xmlns=""http://www.w3.org/1999/xhtml"" sty..."
1,H1NGX3gBPctInV7JRhi5,"<html xmlns=""http://www.w3.org/1999/xhtml"" sty..."
2,IFNGX3gBPctInV7JRhjG,"<html xmlns=""http://www.w3.org/1999/xhtml"" sty..."
3,IVNGX3gBPctInV7JRhjT,"<html xmlns=""http://www.w3.org/1999/xhtml"" sty..."
4,IlNGX3gBPctInV7JRhjd,"<html xmlns=""http://www.w3.org/1999/xhtml"" sty..."
...,...,...
1995,N1NYX3gBPctInV7JziOm,"<html xmlns=""http://www.w3.org/1999/xhtml"" sty..."
1996,OFNYX3gBPctInV7JziO4,"<html xmlns=""http://www.w3.org/1999/xhtml"" sty..."
1997,OVNYX3gBPctInV7JziPC,"<html xmlns=""http://www.w3.org/1999/xhtml"" sty..."
1998,OlNYX3gBPctInV7JziPL,"<html xmlns=""http://www.w3.org/1999/xhtml"" sty..."


In [7]:
uno_html = df.iloc[0]["raw_html"] #first sample html

### Parsing on sample HTML

In [8]:
soup = BeautifulSoup(uno_html, 'html.parser')

In [9]:
first_row = soup

### Building the Parser for meat DF with BeautifulSoup

Testing our extract statements

In [10]:
# final extract statements for each detail with BS4

# company details
company_link = first_row.find('a')['href']
company_phone = first_row.find('a', href="tel:").text


# order details
order_deets = first_row.find('td', class_ = "order-details").text
order_deets = order_deets.split('\n')
order_deets = list(filter(None, order_deets))
order_num = order_deets[0]
order_placed = order_deets[1]
delivery_date = order_deets[2]


# order, shipment and payment information
info_list = []
order_list = []
ship_and_pay = []
order_info = first_row.find_all('tbody')

for deet in order_info:
    d_text = deet.text
    d_text = d_text.split('\n')
    d_text = list(filter(None, d_text))
    d_strip = [d.strip() for d in d_text]
    info_list.append(d_strip)

for listt in info_list:
    if 'Subtotal' not in listt:
        order_list.append(listt)
    else:
        ship_and_pay.append(listt)
        

# address info
addresses = first_row.find_all('td', class_ = "address-details")
bill_address = []
ship_address = []
for adr in addresses:
    adr_text = adr.text
    adr_text = adr_text.split('\n')
    adr_text = list(filter(None, adr_text))
    if 'Bill to:' in adr_text:
        bill_address = adr_text[1:]
    elif 'Ship to:' in adr_text:
        ship_address = adr_text[1:]    
        

# method info
methods = first_row.find_all('td', class_ = "method-info")
for mthd in methods:
    mthd_text = mthd.text
    mthd_text = mthd_text.split('\n')
    mthd_text = list(filter(None, mthd_text))
    if 'Shipping method:' in mthd_text:
        ship_method = mthd_text[1:]
    elif 'Payment method:' in mthd_text:
        pay_method = mthd_text[1:]
    


# printing all extracted data
print(company_link)
print(company_phone)
print(order_num)
print(order_placed)
print(delivery_date)
print(order_list)
print(ship_and_pay)
print(bill_address)
print(ship_address)
print(ship_method)
print(pay_method)

https://www.freshtohome.com/
1800-313-3302
Your order #STHY802943
Placed on March 20, 2021 5:57:25 PM IST
Expected Delivery on Mar 21, 2021 (8:00am to 11:30am)
[['Seer Fish / King Fish / Surmai / Neymeen / Vanjaram / Anjal (2kg to 5kg) - Steak Combo (2x 400g Pack) (2.5kg+)', 'seer-fish-1', '1 pack', '₹859.00'], ['Premium Tender and Antibiotic-residue-free Chicken - Curry Cut (Skinless)', 'chicken-curry-cut', '1 Kg', '₹218.00']]
[['Subtotal', '₹1,077.00', 'Shipping & Handling', '₹29.00', 'Free Shipping', '-₹29.00', 'Grand Total', '₹1,077.00', 'Online Payment', '₹1,077.00', 'Amount Due', '₹0.00']]
['Joyjit Gupta', '#2406 A WING, OMKAR ANANTA', 'FILM CITY ROAD', 'Nagari Niwara', 'Mumbai,  Maharashtra, 400065', 'India', 'T: 9664451060', 'Land Mark: NEXT TO INDIRA GANDHI INSTITUTE']
['Joyjit Gupta', '#2406 A WING, OMKAR ANANTA', 'FILM CITY ROAD', 'Nagari Niwara', 'Mumbai,  Maharashtra, 400065', 'India', 'T: 9664451060', 'Land Mark: NEXT TO INDIRA GANDHI INSTITUTE']
['Home Delivery - Flat']


---

Building the meat Parsed Dataset

In [11]:
records = []

In [12]:
# building above statements into function with if-else logic to handle exceptions
# we exclude the company link and phone number since it'd be the same for the 2000 mails

def extract_info(raw_html):
    global ship_method
    # creating soup object
    souped_html = BeautifulSoup(raw_html, 'html.parser')


    # get order details
    order_deets_test = souped_html.find('td', class_ = "order-details")
    if order_deets_test:
        order_deets = order_deets_test.text
        order_deets = order_deets.split('\n')
        order_deets = list(filter(None, order_deets))
        order_num = order_deets[0]
        order_placed = order_deets[1]
        delivery_date = order_deets[2]
    else:
        order_num = "N/A"
        order_placed = "N/A"
        delivery_date = "N/A"
    

    # get order, shipment and payment information
    info_list = []
    order_list = []
    ship_and_pay = []
    
    order_info_test = souped_html.find_all('tbody')
    if order_info_test:
        order_info = order_info_test
        for deet in order_info:
            d_text = deet.text
            d_text = d_text.split('\n')
            d_text = list(filter(None, d_text))
            d_strip = [d.strip() for d in d_text]
            info_list.append(d_strip)

        for listt in info_list:
            if 'Subtotal' not in listt:
                order_list.append(listt)
            else:
                ship_and_pay.append(listt)
        
    else:
        order_list.append("N/A")
        ship_and_pay.append("N/A")
    
    
    # get address info
    addresses_test = souped_html.find_all('td', class_ = "address-details")
    if addresses_test:
        addresses = addresses_test
        bill_address = []
        ship_address = []
        for adr in addresses:
            adr_text = adr.text
            adr_text = adr_text.split('\n')
            adr_text = list(filter(None, adr_text))
            if 'Bill to:' in adr_text:
                bill_address = adr_text[1:]
            elif 'Ship to:' in adr_text:
                ship_address = adr_text[1:]    
    else:
        bill_address = "N/A"
        ship_address = "N/A"    
        
    
    # get method info
    methods_test = souped_html.find_all('td', class_ = "method-info")
    if methods_test:
        methods = methods_test
        for mthd in methods:
            mthd_text = mthd.text
            mthd_text = mthd_text.split('\n')
            mthd_text = list(filter(None, mthd_text))
            if 'Shipping method:' in mthd_text:
                ship_method = mthd_text[1:]
            elif 'Payment method:' in mthd_text:
                pay_method = mthd_text[1:]
            else:
                ship_method = "N/A"
                pay_method = "N/A"
    else:
        ship_method = "N/A"
        pay_method = "N/A"
        
        
    
    # appending all to records list
    records.append((order_num, order_placed, delivery_date, order_list, ship_and_pay, bill_address, ship_address, ship_method, pay_method))

In [13]:
# applying the extract function to all rawHTMLs

for x in df["raw_html"]:
    extract_info(x)

In [14]:
len(records)

2000

In [15]:
# making a pandas DF out of our generated list

meat_data = pd.DataFrame(records, columns=["order_num", "order_placed", "delivery_date", "order_list", "ship_and_pay", "bill_address", "ship_address", "ship_method", "pay_method"])

In [16]:
meat_data.tail(5)

,order_num,order_placed,delivery_date,order_list,ship_and_pay,bill_address,ship_address,ship_method,pay_method
1995,Your order #STHK1010354,"Placed on January 23, 2021 11:35:45 AM IST","Expected Delivery on Jan 24, 2021 (8:00am - 11...",[[Special Premium Tender and Antibiotic-residu...,"[[Subtotal, ₹379.00, Shipping & Handling, ₹29....","[Vishal Vijay V J, #TC 22 /88 Seetha Bhavan, P...","[Vishal Vijay V J, #TC 22 /88 Seetha Bhavan, P...",[Home Delivery - Flat],[Online Payment]
1996,Your order #STHK1023372,"Placed on January 30, 2021 10:15:13 AM IST","Expected Delivery on Jan 31, 2021 (8:00am - 11...",[[Special Premium Tender and Antibiotic-residu...,"[[Subtotal, ₹529.00, Shipping & Handling, ₹29....","[Vishal Vijay V J, #TC 22 /88 Seetha Bhavan, P...","[Vishal Vijay V J, #TC 22 /88 Seetha Bhavan, P...",[Home Delivery - Flat],[Online Payment]
1997,Your order #STHK1060864,"Placed on February 20, 2021 11:46:09 AM IST","Expected Delivery on Feb 21, 2021 (8:00am - 11...",[[Special Premium Tender and Antibiotic-residu...,"[[Subtotal, ₹529.00, Shipping & Handling, ₹29....","[Vishal Vijay V J, #TC 22 /88 Seetha Bhavan, P...","[Vishal Vijay V J, #TC 22 /88 Seetha Bhavan, P...",[Home Delivery - Flat],[Online Payment]
1998,Your order #STHK1099381,"Placed on March 14, 2021 6:14:12 PM IST","Expected Delivery on Mar 15, 2021 (8:00am - 11...",[[Premium Chicken Dressed without Skin - Curry...,"[[Subtotal, ₹396.00, Shipping & Handling, ₹29....","[Vishal Vijay V J, #TC 22 /88 Seetha Bhavan, P...","[Vishal Vijay V J, #TC 22 /88 Seetha Bhavan, P...",[Home Delivery - Flat],[Online Payment]
1999,Your order #STHB2768003,"Placed on March 25, 2020 2:48:20 PM IST","Expected Delivery on Mar 29, 2020 (7:30am - 10...",[[Premium Tender and Antibiotic-free Chicken -...,"[[Subtotal, ₹327.00, Shipping & Handling, ₹29....","[SUKANTA PAUL, #937 Ground floor 25th cross, 2...","[SUKANTA PAUL, #937 Ground floor 25th cross, 2...",[Home Delivery - Flat],[Online Payment]


### Handling N/A Exceptions

In [17]:
meat_data.describe()

,order_num,order_placed,delivery_date,order_list,ship_and_pay,bill_address,ship_address,ship_method,pay_method
count,2000,2000,2000,2000,2000,2000,2000,2000,2000
unique,1932,1899,1603,1483,1415,630,631,5,6
top,N/A,N/A,N/A,[N/A],[N/A],N/A,N/A,[Home Delivery - Flat],[Online Payment]
freq,59,59,59,59,59,59,59,1360,1551


In [18]:
meat_notna = meat_data[meat_data['order_num'] != 'N/A']

In [19]:
meat_notna.describe()

,order_num,order_placed,delivery_date,order_list,ship_and_pay,bill_address,ship_address,ship_method,pay_method
count,1941,1941,1941,1941,1941,1941,1941,1941,1941
unique,1931,1898,1602,1482,1414,629,630,4,5
top,Your order #STHB2960996,"Placed on April 24, 2020 1:15:51 PM IST","Expected Delivery on May 30, 2020 (8:00am to 0...",[[Premium Tender and Antibiotic-free Chicken -...,"[[Subtotal, ₹218.00, Shipping & Handling, ₹29....","[Sathish Kumar.J, Christina nilayam,2nd floor,...","[Sathish Kumar.J, Christina nilayam,2nd floor,...",[Home Delivery - Flat],[Online Payment]
freq,3,3,6,41,42,42,42,1360,1551


---

## 2. Report after parsing meat data

**Out of a total of 2000 mails, 1941 of them were parsed successfully. 59 Mails were from the Customer Care to individuals, containing instructions to complete their payment. They had no real information, so no valuable data was lost. Hence, 97% of the data was parsed successfully.**

### Printing parsed meat data to CSV

In [20]:
# exporting the parsed data to CSV

meat_notna.to_csv('parsedData_meat.csv', index=False, encoding="utf-8")

## 3. Cleaning the parsed meat data

In [21]:
meat_cleaned = pd.DataFrame()

In [22]:
meat_notna.sample(2)

,order_num,order_placed,delivery_date,order_list,ship_and_pay,bill_address,ship_address,ship_method,pay_method
262,Your order #STHY459706,"Placed on July 18, 2020 6:59:44 PM IST","Expected Delivery on Jul 19, 2020 (8:00am to 0...",[[Special Premium Tender and Antibiotic-free S...,"[[Subtotal, ₹499.00, Shipping & Handling, ₹29....","[SHANKAR, #11 ENGINEER BULDING, AIR FORCE QUAR...","[SHANKAR, #11 ENGINEER BULDING, AIR FORCE QUAR...",[Home Delivery - Flat],[Online Payment]
1782,Your order #STHP152265,"Placed on November 30, 2019 5:48:39 PM IST","Expected Delivery on Dec 1, 2019 (8am - 11am)",[[Premium Tender and Antibiotic-free Chicken -...,"[[Subtotal, ₹297.00, Shipping & Handling, ₹29....","[Apoorv Anand, Flat 302, palace paradise , ram...","[Apoorv Anand, Flat 302, palace paradise , ram...",[Home Delivery - Flat],[Online Payment]


We have to clean all the columns separately and even split the features for some of them.

### Cleaning order_num column

In [23]:
import re

In [24]:
order_num_regex = re.compile(r'#(\w+)', re.IGNORECASE)
mo = order_num_regex.search('Your order #E09D19870')
print(mo.group(1))

E09D19870


In [25]:
# function to remove the words and #

def clean_ordernum(x):
    order_num_regex = re.compile(r'#(\w+)', re.IGNORECASE)
    xx = order_num_regex.search(x)
    num_new = xx.group(1)
    return num_new

meat_cleaned["order_num"] = meat_notna["order_num"].map(lambda x: clean_ordernum(x))

In [26]:
meat_cleaned.sample(10)

,order_num
230,E01C16277
781,STHC119032
1565,STHB1476967
376,STHB3673618
194,STHP178706
630,STHB2082449
1598,STHB2399608
1735,STHD575630
392,E04B50740
692,E03B76750


### Cleaning order_placed column

In [27]:
meat_notna["order_placed"].sample(5)

506      Placed on January 30, 2021 5:27:48 PM IST
354         Placed on March 4, 2020 5:02:15 PM IST
1961    Placed on December 14, 2020 3:06:24 PM IST
986     Placed on November 28, 2020 3:34:36 PM IST
25       Placed on December 1, 2020 6:20:49 PM IST
Name: order_placed, dtype: object

In [28]:
order_placed_regex = re.compile(r'on (.+)\s', re.IGNORECASE)
mo = order_placed_regex.search('Placed on February 21, 2018 3:26:24 PM IST')
zz = mo.group(1)
print(mo.group(1))

February 21, 2018 3:26:24 PM


In [29]:
pd.to_datetime(zz)

Timestamp('2018-02-21 15:26:24')

In [30]:
# function to remove the words and time sonze

def clean_orderplaced(x):
    order_placed_regex = re.compile(r'on (.+)\s', re.IGNORECASE)
    xx = order_placed_regex.search(x)
    date_new = xx.group(1)
    return date_new

meat_cleaned["order_placed"] = meat_notna["order_placed"].map(lambda x: clean_orderplaced(x))

In [31]:
# converting to a datetime object

meat_cleaned["order_placed"] = pd.to_datetime(meat_cleaned["order_placed"])
meat_cleaned.sample(10)

,order_num,order_placed
49,STHB4148606,2020-12-28 16:47:49
1729,STHD639606,2019-08-02 17:56:08
1054,STHP364391,2021-01-25 10:55:11
1439,STHD697114,2019-09-07 20:27:48
1850,E08D9178,2020-01-11 19:21:11
454,STHY264806,2020-04-07 00:11:05
1096,E06B11472,2019-06-27 10:22:05
1927,STHK922913,2020-12-07 17:03:10
212,STHP367842,2021-01-31 18:04:47
966,STHP224031,2020-05-22 11:15:58


### Cleaning delivery_date into date column

In [32]:
meat_notna.sample(2)

,order_num,order_placed,delivery_date,order_list,ship_and_pay,bill_address,ship_address,ship_method,pay_method
1058,Your order #STHP389637,"Placed on March 19, 2021 6:34:09 PM IST","Expected Delivery on Mar 20, 2021 (8am -11am)","[[Premium Sardine / Mathi (Soft, fish belly mi...","[[Subtotal, ₹318.00, Shipping & Handling, ₹0.0...","[Antony Thomas, #C1-101 Manjri greens annex, H...","[Antony Thomas, #C1-101 Manjri greens annex, H...",[Subscription - Free shipping],[Online Payment]
971,Your order #STHH66594,"Placed on December 12, 2020 1:49:50 PM IST","Expected Delivery on Dec 13, 2020 (4:30pm to 7...",[[Triple Combo Pack (500g Premium Goat Curry C...,"[[Subtotal, ₹575.00, Shipping & Handling, ₹29....","[Pallavi Yadav, #4-3-483/ Residential, Bank St...","[Pallavi Yadav, #4-3-483/ Residential, Bank St...",[Home Delivery - Flat],[Cash On Delivery]


In [33]:
meat_notna.iloc[618]["delivery_date"]

'Expected Delivery on Dec 29, 2018 Between (4:00pm - 8:30pm)'

In [34]:
deliverydate_regex = re.compile(r'on (.+)\sbetween', re.IGNORECASE)
date_res = deliverydate_regex.search('Expected Delivery on Dec 29, 2018 Between (4:00pm - 8:30pm)')
print(date_res.group(1))

Dec 29, 2018


In [35]:
# function to remove words and split the field into date

def clean_delivery_date(x):
    delivery_date_regex = re.compile(r'on (.+, \d{4})\s', re.IGNORECASE)
    date_res = delivery_date_regex.search(x)
    date_new = date_res.group(1)
    return date_new

meat_cleaned["delivery_date"] = meat_notna["delivery_date"].map(lambda x: clean_delivery_date(x))

In [36]:
meat_cleaned.sample(5)

,order_num,order_placed,delivery_date
588,STHD328590,2018-10-16 13:07:53,"Oct 17, 2018"
326,STHD1387969,2020-09-23 17:54:35,"Sep 23, 2020"
1088,STHB3501014,2020-07-11 15:44:25,"Jul 12, 2020"
1162,E03B77152,2021-01-23 12:40:48,"Jan 23, 2021"
560,STHB4110440,2020-12-19 19:47:04,"Dec 20, 2020"


### Cleaning delivery_date into time details column

In [37]:
meat_notna.iloc[1779]["delivery_date"]

'Expected Delivery on Aug 11, 2020 (6am - 1:30pm)'

In [38]:
# function to remove words and split the field into time

def clean_delivery_details(x):
    delivery_details_reg = re.compile(r', \d{4}\s\((.+)\)', re.IGNORECASE)
    det_res = delivery_details_reg.search(x)
    if det_res:
        det_new = det_res.group(1)
        return det_new
    else:
        return ""

meat_cleaned["delivery_details"] = meat_notna["delivery_date"].map(lambda x: clean_delivery_details(x))

In [39]:
meat_cleaned.sample(5)

,order_num,order_placed,delivery_date,delivery_details
714,STHB3505237,2020-07-11 21:55:58,"Jul 12, 2020",3:00pm - 6:00pm
1323,E06C11535,2020-04-21 10:23:51,"Apr 21, 2020",2 hours - Express Delivery
1296,E01K39895,2021-02-24 11:49:08,"Feb 24, 2021",Express Delivery - 90 Minutes
1351,STHK940982,2020-12-17 20:01:57,"Dec 18, 2020",11:00am - 01:00pm
1705,STHY342126,2020-05-29 09:02:04,"May 30, 2020",8:00am to 01:00pm


### Splitting order_list features column

In [40]:
meat_data["order_list"].sample(3)

761     [[Premium Tender and Antibiotic-free Chicken -...
1815    [[Premium Sardine / Mathi (Soft, fish belly mi...
175     [[Fresh Indian Baasa / Pangasius - Boneless Fi...
Name: order_list, dtype: object

In [41]:
sample_order = meat_data.iloc[11]["order_list"]
sample_order

[['Premium Tender and Antibiotic-residue free Chicken - Curry Cut (Skinless)',
  'chicken-curry-cut',
  '1.5 Kg',
  '₹447.00'],
 ['Pabda / Pabda catfish / পাবদা- Whole (Not Cleaned, Not Gutted) (8 to 15 Count/kg)',
  'pabda',
  '0.5 Kg',
  '₹399.00']]

In [42]:
split_table = []

In [43]:
# function to split order list into the table items

def split_order_table(x):
    item_list = []
    sku_list = []
    qty_list = []
    subtotal_list = []

    for listt in x:
        item_list.append(listt[0])
        sku_list.append(listt[1])
        qty_list.append(listt[2])
        subtotal_list.append(listt[3])
        
    split_table.append((item_list, sku_list, qty_list, subtotal_list))

In [44]:
for x in meat_notna["order_list"]:
    split_order_table(x)

In [45]:
split_order_data = pd.DataFrame(split_table, columns=["item_list", "sku_list", "qty_list", "subtotal_list"])
split_order_data.sample(3)

,item_list,sku_list,qty_list,subtotal_list
1572,[Premium Tender and Antibiotic Free Chicken - ...,[chicken-curry-cut],[1.5 kg],[₹238.50]
1412,[Premium Antibiotic-free Chicken Breast Fillet...,[chicken-fillet],[0.5 kg],[₹199.00]
967,[Premium Tender and Antibiotic-free Chicken - ...,[chicken-curry-cut],[1 Kg],[₹238.00]


In [46]:
meat_cleaned["item"] = split_order_data["item_list"]
meat_cleaned["sku"] = split_order_data["sku_list"]
meat_cleaned["qty"] = split_order_data["qty_list"]
meat_cleaned["subtotal"] = split_order_data["subtotal_list"]

In [47]:
meat_cleaned.sample(4)

,order_num,order_placed,delivery_date,delivery_details,item,sku,qty,subtotal
334,STHP317974,2020-10-12 08:37:38,"Oct 13, 2020",8am -11am,[Rohu - Curry cut (May include head pieces) (1...,"[rohu, chicken-curry-cut]","[1 kg, 1 kg]","[Rs. 148.96, Rs. 198.00]"
1682,STHB4056825,2020-12-08 18:44:54,"Dec 9, 2020",7:30am - 9:30am,"[Tangy Chicken Ghee Roast (250g pack), Premium...","[chicken-ghee-roast-250g, chicken-curry-cut]","[2 pack, 0.5 Kg]","[₹250.00, ₹109.00]"
1362,STHH75726,2020-12-30 09:49:43,"Dec 31, 2020",7:30am to 11:30am,[Premium Tender and Antibiotic-residue-free Ch...,"[chicken-curry-cut, chicken-whole-leg-pack-2]","[0.5 Kg, 1 pack]","[₹144.50, ₹169.00]"
17,STHD1403145,2020-09-30 10:53:52,"Oct 1, 2020",7am - 11:30am,[Silver Pomfret / Avoli (100g to 200g) - Whole...,"[silver-pomfret-medium, indian-prawns, mud-cra...","[0.5 kg, 0.6 kg, 0.5 kg]","[₹449.00, ₹816.60, ₹424.00]"


### Cleaning subtotal column

We are not cleaning qty since it contains quantities in both Kg and packs.

In [48]:
meat_cleaned.subtotal.sample(5)

695     [Rs37.00, Rs30.00, Rs29.50, Rs77.50]
764                                [₹198.00]
1397                               [₹278.00]
593                        [₹575.00, ₹59.00]
257                                [₹387.00]
Name: subtotal, dtype: object

In [49]:
# function to convert currency with a comma to a float number

def remove_sign(x):
    from re import sub
    
    new_money = []
    if type(x) == float:
        return x
    
    if len(x) > 1:
        for money in x: 
            value = re.sub(r'[^\d.]', '', money)
            if value:
                new_money.append(value)
            else:
                new_money.append("")
    else:
        value = re.sub(r'[^\d.]', '', str(x))
        if value:
            new_money.append(value)
        else:
            new_money.append("")

    return new_money

In [50]:
meat_cleaned["subtotal"] = meat_cleaned["subtotal"].map(lambda x: remove_sign(x))

In [51]:
meat_cleaned.sample(5)

,order_num,order_placed,delivery_date,delivery_details,item,sku,qty,subtotal
320,STHY413467,2020-07-02 18:06:17,"Jul 3, 2020",8:00am to 01:00pm,[Premium Tender Goat - Curry Cut (include bone...,"[goat-curry-cut, chicken-curry-cut, chicken-dr...","[0.5 kg, 1 Kg, 0.5 kg]","[499.00, 288.00, 199.00]"
491,E04D9718,2019-08-10 20:09:46,"Aug 10, 2019",2 hours - Express Delivery,[Anchovy / Natholi (Medium) - Whole cleaned (S...,"[anchovy-medium, chicken-biryani-cut]","[0.5 kg, 1.1 kg]","[215.74, 261.80]"
226,E10B50557,2021-01-31 11:47:03,"Jan 31, 2021",90 Minutes - Express Delivery,[Premium Sardine / Mathi - Whole Cleaned (20+ ...,"[special-sardine, malabar-sambar-cut-vegetable...","[0.5 kg, 1 pack]","[190.08, 61.00]"
1824,STHY558758,2020-09-26 19:44:57,"Sep 27, 2020",8:00am to 11:30am,[Kerala Sardine / Naadan Mathi - Whole Cleaned...,"[kerala-sardine-1, clam-meat-large, grey-mullet]","[0.5 Kg, 0.5 Kg, 0.5 kg]","[209.00, 119.00, 290.50]"
1752,E02K25972,2020-09-29 13:26:27,"Sep 29, 2020",90 Minutes - Express Delivery,"[Premium Chicken Breast Fillet (Boneless cut),...","[chicken-fillet, baasa]","[0.5 kg, 1 pack]","[199.00, 142.20]"


In [52]:
meat_cleaned.describe()

C:\Users\KIIT\AppData\Local\Temp/ipykernel_13980/3897193054.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  meat_cleaned.describe()


,order_num,order_placed,delivery_date,delivery_details,item,sku,qty,subtotal
count,1941,1941,1941,1941,1884,1884,1884,1884
unique,1931,1898,694,78,1109,844,314,1099
top,STHB2960996,2020-04-24 13:15:51,"Feb 14, 2021",2 hours - Express Delivery,[Premium Tender and Antibiotic-free Chicken - ...,[chicken-curry-cut],[1 Kg],[218.00]
freq,3,3,15,194,192,353,252,88
first,NaN,2018-01-03 12:45:52,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,2021-03-21 22:00:23,NaN,NaN,NaN,NaN,NaN,NaN


### Cleaning ship_and_pay column

In [53]:
meat_notna.sample(3)

,order_num,order_placed,delivery_date,order_list,ship_and_pay,bill_address,ship_address,ship_method,pay_method
1663,Your order #STHB2816967,"Placed on April 4, 2020 2:45:53 PM IST","Expected Delivery on Apr 5, 2020 (3:00pm - 7:0...",[[Premium Tender and Antibiotic-free Chicken -...,"[[Subtotal, ₹769.00, Shipping & Handling, ₹29....","[Abhishek, No 68 5th 5th cross krishna layout ...","[Abhishek, No 68 5th 5th cross krishna layout ...",[Home Delivery - Flat],[Online Payment]
1934,Your order #STHB3243674,"Placed on June 1, 2020 7:54:46 AM IST","Expected Delivery on Jun 1, 2020 (3:00pm - 6:0...",[[Mix n Match Fish Combo (Small) - Whole (10g ...,"[[Subtotal, ₹674.00, Shipping & Handling, ₹29....","[RUDRANJAN SAHA, #56 56, Kakade nivas, 5th mai...","[RUDRANJAN SAHA, #56 56, Kakade nivas, 5th mai...",[Home Delivery - Flat],[Online Payment]
1692,Your order #STHC526714,"Placed on February 6, 2021 6:28:17 PM IST","Expected Delivery on Feb 7, 2021 (9:30am to 11...",[[Triple Combo Pack (500g Premium Goat Curry C...,"[[Subtotal, ₹525.00, Shipping & Handling, ₹29....","[Valentinoabin, 100/50 west jones road, Jones ...","[Valentinoabin, 100/50 west jones road, Jones ...",[Home Delivery - Flat],[Online Payment]


In [54]:
meat_notna.ship_and_pay.sample(5)

1217    [[Subtotal, ₹648.00, Shipping & Handling, ₹29....
1743    [[Subtotal, ₹629.00, Shipping & Handling, ₹29....
706     [[Subtotal, ₹998.00, Shipping & Handling, ₹29....
1832    [[Subtotal, ₹218.00, Shipping & Handling, ₹29....
162     [[Subtotal, ₹357.00, Shipping & Handling, ₹0.0...
Name: ship_and_pay, dtype: object

In [55]:
sample_ship = meat_notna.iloc[528].ship_and_pay
sample_ship

[['Subtotal',
  '₹493.90',
  'Shipping & Handling',
  '₹29.00',
  'Grand Total',
  '₹522.90',
  'Online Payment',
  '₹522.90',
  'Amount Due',
  '₹0.00']]

In [56]:
# converting our list to a dictionary which we can split into columns later on 

def convert_dict(b):
    for a in b:
        it = iter(a)
        res_dct = dict(zip(it, it))
        return res_dct
         
meat_cleaned["ship_dict"] = meat_notna["ship_and_pay"].map(lambda x: convert_dict(x))

In [57]:
meat_cleaned.sample(5)

,order_num,order_placed,delivery_date,delivery_details,item,sku,qty,subtotal,ship_dict
1373,STHK990268,2021-01-11 20:37:42,"Jan 12, 2021",11:00am - 01:00pm,[Premium Tender and Antibiotic-residue-free Ch...,[chicken-curry-cut],[1 Kg],[218.00],"{'Subtotal': '₹693.00', 'Shipping & Handling':..."
1183,E01K17735,2019-10-27 09:25:34,"Oct 27, 2019",90 minutes,[Granny's Masala Fried Chicken - 500g Pack],[granny-chicken-500],[1 pack],[245.00],"{'Subtotal': '₹594.00', 'Shipping & Handling':..."
1744,STHB3236211,2020-05-30 19:24:42,"May 31, 2020",7:30am - 11:59am,[Premium Chicken Drumsticks (Free from all gro...,[chicken-drumsticks],[0.5 kg],[199.00],"{'Subtotal': '₹258.00', 'Shipping & Handling':..."
226,E10B50557,2021-01-31 11:47:03,"Jan 31, 2021",90 Minutes - Express Delivery,[Premium Sardine / Mathi - Whole Cleaned (20+ ...,"[special-sardine, malabar-sambar-cut-vegetable...","[0.5 kg, 1 pack]","[190.08, 61.00]","{'Subtotal': '₹447.00', 'Shipping & Handling':..."
1868,STHP179458,2020-02-22 11:29:50,"Feb 22, 2020",4:00pm - 7:30pm,[Premium Antibiotic-residue-free Chicken Drums...,"[chicken-drumsticks-packof-6, chicken-curry-cut]","[2 pack, 0.5 Kg]","[398.00, 129.00]","{'Subtotal': '₹324.00', 'Shipping & Handling':..."


In [58]:
# splitting the dict columns into new columns in a separate DF
df2 = pd.json_normalize(meat_cleaned['ship_dict'])

In [59]:
df2.describe()

,Subtotal,Shipping & Handling,Free Shipping,Grand Total,Online Payment,Amount Due,Tax,Free shipping,Discount,Store Credits,...,"FLAT 10% OFF on orders above 399/-, Free shipping",75/- Off + Guaranteed 75/- CashBack,Congratulations! FREE Fish Cutlets for you because you're a VIP!,Free Shipping For New User,"Free Chettinad Chicken, Free Shipping",Flat 50% off on your next order above Rs. 299/- (upto 250/-),20% discount (UPTO 60) + Guaranteed 20% cashback (UPTO 60),"Guaranteed 100/- cashback on orders above 499, Free Shipping","Referee Discount - 20% savings (10% OFF + 10% Cashback) capped at Rs. 100 each, Free Shipping","FREE Chicken Cutlets for you!, Free Shipping"
count,1941,1940,172,1941,1561,1941,175,128,162,27,...,1,1,1,1,1,1,1,1,1,1
unique,877,12,1,1033,913,261,105,3,44,19,...,1,1,1,1,1,1,1,1,1,1
top,₹218.00,₹29.00,-₹29.00,₹247.00,₹247.00,₹0.00,₹15.00,-₹29.00,-₹29.00,₹200.00,...,-₹122.80,-₹75.00,-₹125.00,-₹29.00,-₹178.00,-₹250.00,-₹60.00,-₹29.00,-₹111.60,-₹154.00
freq,90,1630,172,62,42,1545,10,119,94,4,...,1,1,1,1,1,1,1,1,1,1


In [60]:
# adding only these features since they have the most value counts

meat_cleaned["order_total"] = df2["Subtotal"]
meat_cleaned["shipping_and_handling"] = df2["Shipping & Handling"]
meat_cleaned["Grand Total"] = df2["Grand Total"]
meat_cleaned["Online Payment"] = df2["Online Payment"]
meat_cleaned["Amount Due"] = df2["Amount Due"]

In [61]:
meat_cleaned.sample(5)

,order_num,order_placed,delivery_date,delivery_details,item,sku,qty,subtotal,ship_dict,order_total,shipping_and_handling,Grand Total,Online Payment,Amount Due
252,STHD155595,2018-01-20 17:28:16,"Jan 21, 2018",,[Premium Chicken - 1 Full Chicken Curry Cut (S...,[chicken-curry-cut],[2 kg],[300.00],"{'Subtotal': '₹303.86', 'Shipping & Handling':...",Rs300.00,Rs29.00,Rs329.00,Rs329.00,Rs0.00
337,STHB1420111,2018-12-29 18:04:51,"Dec 30, 2018",,[Premium Tender and Antibiotic Free Chicken - ...,[chicken-curry-cut],[1.5 kg],[297.00],"{'Subtotal': '₹199.00', 'Shipping & Handling':...",₹297.00,₹29.00,₹326.00,NaN,₹326.00
687,STHC246415,2020-03-02 14:00:05,"Mar 2, 2020",5:00pm - 8:30pm,[Special Premium Tender and Antibiotic-free Sk...,[chicken-curry-cut-2kg-pack],[1 pack],[555.00],"{'Subtotal': '₹424.00', 'Shipping & Handling':...",₹555.00,₹29.00,₹584.00,₹584.00,₹0.00
1027,STHP147689,2019-11-19 16:39:17,"Nov 20, 2019",4:00pm - 7:30pm,[Mackerel / Ayala / Bangda / Aylai (10 to 14 C...,[indian-mackerel],[1 Kg],[278.00],"{'Subtotal': '₹278.00', 'Shipping & Handling':...",₹278.00,₹0.00,₹278.00,₹278.00,₹0.00
752,STHY168482,2019-11-22 18:42:08,"Nov 24, 2019",8:00am - 11:59am,[Premium Tender and Antibiotic-free Chicken - ...,[chicken-curry-cut],[3 Kg],[594.00],"{'Subtotal': '₹239.00', 'Shipping & Handling':...",₹594.00,₹29.00,₹594.00,₹594.00,₹0.00


In [62]:
meat_cleaned.drop(meat_cleaned.columns[[8]], axis = 1, inplace=True)

In [63]:
meat_cleaned.sample(3)

,order_num,order_placed,delivery_date,delivery_details,item,sku,qty,subtotal,order_total,shipping_and_handling,Grand Total,Online Payment,Amount Due
602,STHY228865,2020-02-15 05:37:26,"Feb 16, 2020",8:00am - 11:59am,[Premium Tender Goat - Curry Cut (include bone...,"[goat-curry-cut, bombay-duck]","[0.5 kg, 0.5 Kg]","[349.00, 119.00]",₹468.00,₹29.00,₹497.00,₹497.00,₹0.00
247,E01C29244,2020-08-07 22:51:52,"Aug 8, 2020",8:00am - 12:30pm,[Premium Tender and Antibiotic-free Chicken - ...,[chicken-curry-cut],[1.5 kg],[297.00],₹297.00,₹29.00,₹207.90,₹207.90,₹0.00
707,STHD1565988,2020-12-17 22:14:59,"Dec 18, 2020",7am - 11:30am,[Premium Antibiotic-free Chicken Dressed with ...,[chicken-with-skin],[1.2 kg],[273.60],₹273.60,₹29.00,₹273.60,NaN,₹273.60


In [64]:
# function to remove the currency sign from the recently split columns

def sign_remove(x):
    sign_reg = re.compile(r'(\d.+)', re.IGNORECASE)
    sign_res = sign_reg.search(str(x))
    if sign_res:
        sign_new = sign_res.group(1)
        return sign_new
    else:
        return x

In [65]:
meat_cleaned["order_total"] = meat_cleaned["order_total"].map(lambda x: sign_remove(x))
meat_cleaned["shipping_and_handling"] = meat_cleaned["shipping_and_handling"].map(lambda x: sign_remove(x))
meat_cleaned["Grand Total"] = meat_cleaned["Grand Total"].map(lambda x: sign_remove(x))
meat_cleaned["Online Payment"] = meat_cleaned["Online Payment"].map(lambda x: sign_remove(x))
meat_cleaned["Amount Due"] = meat_cleaned["Amount Due"].map(lambda x: sign_remove(x))

In [66]:
meat_cleaned.sample(3)

,order_num,order_placed,delivery_date,delivery_details,item,sku,qty,subtotal,order_total,shipping_and_handling,Grand Total,Online Payment,Amount Due
298,STHD1185148,2020-06-20 22:36:24,"Jun 21, 2020",6am - 1:30pm,[Premium Tender and Antibiotic-residue-free Ch...,[chicken-curry-cut],[1 Kg],[238.00],238.00,29.00,267.00,267.00,0.00
1213,STHB3489992,2020-07-10 15:19:43,"Jul 10, 2020",240 mins - Express Delivery,"[Mutton Soup Bones, Preservative Free Malabar ...","[mutton-soup-bones, chicken-pickle-150]","[0.5 Kg, 1 pack]","[299.00, 199.00]",498.00,29.00,550.88,550.88,0.00
1695,STHB1229714,2018-10-01 21:26:48,"Oct 2, 2018",,[Premium Antibiotic-free Chicken Thigh (Bonele...,[chicken-thigh-boneless1],[1 Kg],[450.00],450.00,29.00,389.00,389.00,0.00


### Cleaning bill_address and ship_address column

In [67]:
meat_notna.sample(3)

,order_num,order_placed,delivery_date,order_list,ship_and_pay,bill_address,ship_address,ship_method,pay_method
178,Your order #E04C2615,"Placed on May 22, 2019 11:04:25 AM IST","Expected Delivery on May 22, 2019 (2 hours - E...","[[Lamb - Curry Cut (may include bones), lamb-c...","[[Subtotal, ₹408.00, Shipping & Handling, ₹29....","[Fathima Feroz, C1 F2 VGN IMPARIA PHASE 3, MAH...","[Fathima Feroz, C1 F2 VGN IMPARIA PHASE 3, MAH...",[Home Delivery - Home Delivery],[Cash On Delivery]
706,Your order #STHD1544921,"Placed on December 8, 2020 11:13:10 AM IST","Expected Delivery on Dec 9, 2020 (7am - 11:30am)",[[Special Premium Tender and Antibiotic-residu...,"[[Subtotal, ₹998.00, Shipping & Handling, ₹29....","[Pranav Pramod, #2291 Sec 55, Housing board co...","[Pranav Pramod, #2291 Sec 55, Housing board co...",[Home Delivery - Flat],[Online Payment]
1468,Your order #STHB3946660,"Placed on November 12, 2020 11:22:37 PM IST","Expected Delivery on Nov 13, 2020 (7:30am - 9:...","[[Malabar Chicken Samosas - Pack of 4, chicken...","[[Subtotal, ₹1,339.00, Shipping & Handling, ₹2...","[Mohammad Adil, #70/2 Rk nivas, 6th main 1st c...","[Mohammad Adil, #70/2 Rk nivas, 6th main 1st c...",[Home Delivery - Flat],[Online Payment]


In [68]:
meat_notna.bill_address.sample(5)

1399    [resham Singh, 114 Top floor , AA block, shali...
213     [Subhendu Rana, #503 Tower 8, Blue Ridge towns...
645     [Shibu Thekkadavan, J203, Confident Atik, Somp...
1018    [Sandhya Deepak, #G4 16/35 Sundaram Flats, Ved...
1271    [Richard Gomes, 1-B wing, #207 1-B wing, Walch...
Name: bill_address, dtype: object

In [69]:
sample_adr = meat_notna.iloc[279].ship_address
sample_adr

['Anurag Ranjan',
 '#E1906 Prateek Wisteria',
 'Sector 77',
 'Sarfabad',
 'Noida,  Uttar Pradesh, 201301',
 'India',
 'T: 9971772352']

In [70]:
" ".join(sample_adr)

'Anurag Ranjan #E1906 Prateek Wisteria Sector 77 Sarfabad Noida,  Uttar Pradesh, 201301 India T: 9971772352'

In [71]:
# concatenating strings in a list for one address string

meat_cleaned["billing_address"] = meat_notna["bill_address"].map(lambda x: " ".join(x))
meat_cleaned["shipping_address"] = meat_notna["ship_address"].map(lambda x: " ".join(x))

In [72]:
meat_cleaned.sample(5)

,order_num,order_placed,delivery_date,delivery_details,item,sku,qty,subtotal,order_total,shipping_and_handling,Grand Total,Online Payment,Amount Due,billing_address,shipping_address
439,E17B13403,2020-11-19 10:33:39,"Nov 19, 2020",90 Minutes - Express Delivery,[Premium Tender and Antibiotic-residue-free Ch...,[chicken-curry-cut],[2 Kg],[436.00],436.00,0.00,436.00,436.00,0.00,"Rakesh Menon D.no.1125, 1st floor, 43rd cross...","Rakesh Menon D.no.1125, 1st floor, 43rd cross..."
1591,STHB2104972,2019-09-21 07:16:33,"Sep 21, 2019",4:00pm to 7:30pm,[Country Chicken / Desi Chicken / Natti Koli W...,[country-chicken-skin-curry-cut-pack-small],[1 pack],[499.00],499.00,29.00,528.00,528.00,0.00,"Akshay Koshy Kizhekekara house,No 32, 2nd Floo...","Akshay Koshy Kizhekekara house,No 32, 2nd Floo..."
1831,E06Y2637,2020-10-14 14:29:05,"Oct 14, 2020",3 hours - Express Delivery,[Combo Pack (500g Premium Goat Curry Cut + 500...,[500g-premium-goat-500g-chicken-250g-small-cle...,[1 pack],[575.00],575.00,29.00,529.00,NaN,529.00,"Bhavesh Khomne D, #Room No 1 D, Deepak Industr...","Bhavesh Khomne D, #Room No 1 D, Deepak Industr..."
742,STHD1287585,2020-08-05 14:53:58,"Aug 6, 2020",6am - 1:30pm,[Premium Tender and Antibiotic-free Chicken - ...,"[chicken-curry-cut, queen-fish]","[3 Kg, 1 kg]","[684.00, 551.00]","1,235.00",29.00,988.00,988.00,0.00,"Saurabh A-136 4thfloor Adjacent goel clinic, D...","Saurabh A-136 4thfloor Adjacent goel clinic, D..."
1046,STHP315964,2020-10-08 23:13:23,"Oct 9, 2020",8am -11am,"[FTH VIP - 3 Month Plan, Premium Tender and An...","[fth-vip-3-month, chicken-curry-cut]","[1, 1 Kg]","[0.00, 218.00]",218.00,99.00,317.00,NaN,267.00,Antony Thomas #C1-101 Manjri greens annex Hada...,Antony Thomas #C1-101 Manjri greens annex Hada...


### Adding method columns lastly

In [73]:
meat_notna.ship_method.sample(5)

730     [Home Delivery - Home Delivery]
1712             [Home Delivery - Flat]
293              [Home Delivery - Flat]
1424    [Home Delivery - Home Delivery]
1667             [Home Delivery - Flat]
Name: ship_method, dtype: object

In [74]:
# converting lists to strings

meat_cleaned["shipment_method"] = meat_notna["ship_method"].map(lambda x: " ".join(x))
meat_cleaned["payment_method"] = meat_notna["pay_method"].map(lambda x: " ".join(x))

In [75]:
meat_cleaned.sample(5)

,order_num,order_placed,delivery_date,delivery_details,item,sku,qty,subtotal,order_total,shipping_and_handling,Grand Total,Online Payment,Amount Due,billing_address,shipping_address,shipment_method,payment_method
710,STHY171674,2019-11-26 19:49:48,"Nov 28, 2019",8:00am - 11:59am,[Tilapia / Jalebi Fish - Whole cleaned (90g to...,"[tilapia, idly-batter]","[0.5 Kg, 1 pack]","[161.50, 55.00]",216.50,29.00,248.25,NaN,248.25,"Vishal Gurav 702 Laxmi Narayan CHS , Parel Vil...","Vishal Gurav 702 Laxmi Narayan CHS , Parel Vil...",Home Delivery - Flat,Online Payment
607,E03Y1608,2020-07-15 10:05:40,"Jul 15, 2020",3 hours - Express Delivery,[Special Premium Tender and Antibiotic-residue...,[chicken-curry-cut-2kg-pack],[1 pack],[379.00],379.00,29.00,408.00,408.00,0.00,"VAISHALI Hatkar Room No 9044, #Bld.No 231 Room...","VAISHALI Hatkar Room No 9044, #Bld.No 231 Room...",Home Delivery - Home Delivery,Online Payment
1702,STHB2756448,2020-03-24 10:46:17,"Mar 25, 2020",7:30am - 10:30am,[Premium Antibiotic-free Chicken Breast Fillet...,"[chicken-fillet, combo-1kg-chicken-cut-500g-ma...","[1 kg, 1 pack, 1 pack]","[498.00, 599.00, 207.00]","1,304.00",29.00,"1,328.84","1,328.84",0.00,Rintu Mukherjee 102 Vanshee Solitaire 4th main...,Rintu Mukherjee 102 Vanshee Solitaire 4th main...,Home Delivery - Flat,Online Payment
467,STHC172151,2019-10-26 18:45:29,"Oct 27, 2019",8:00am - 12:30pm,[Premium Tender and Antibiotic-free Chicken / ...,"[chicken-curry-cut, chicken-drumsticks]","[0.5 Kg, 0.5 kg]","[119.00, 175.00]",294.00,0.00,294.00,294.00,0.00,"Subani F12, Aditya Plaza,Old Township Road Nea...","Subani F12, Aditya Plaza,Old Township Road Nea...",Home Delivery - Flat,Online Payment
108,STHD1423341,2020-10-09 11:03:37,"Oct 9, 2020",4:30 - 8:30pm,"[Premium Sardine / Mathi (Soft, fish belly mig...","[special-sardine, yellowfin-tuna]","[1.5 kg, 0.5 Kg]","[597.00, 304.00]",901.00,29.00,901.00,NaN,901.00,Sijo #T 13 505 Exotica Dreamvile Exotica Dream...,Sijo #T 13 505 Exotica Dreamvile Exotica Dream...,Home Delivery - Flat,Cash On Delivery


In [76]:
meat_cleaned.describe()

C:\Users\KIIT\AppData\Local\Temp/ipykernel_13980/3897193054.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  meat_cleaned.describe()


,order_num,order_placed,delivery_date,delivery_details,item,sku,qty,subtotal,order_total,shipping_and_handling,Grand Total,Online Payment,Amount Due,billing_address,shipping_address,shipment_method,payment_method
count,1941,1941,1941,1941,1884,1884,1884,1884,1884,1883,1884,1512,1884,1941,1941,1941,1941
unique,1931,1898,694,78,1109,844,314,1099,845,9,996,883,251,629,630,4,5
top,STHB2960996,2020-04-24 13:15:51,"Feb 14, 2021",2 hours - Express Delivery,[Premium Tender and Antibiotic-free Chicken - ...,[chicken-curry-cut],[1 Kg],[218.00],218.00,29.00,247.00,247.00,0.00,"Sathish Kumar.J Christina nilayam,2nd floor,17...","Sathish Kumar.J Christina nilayam,2nd floor,17...",Home Delivery - Flat,Online Payment
freq,3,3,15,194,192,353,252,88,89,1615,62,42,1517,42,42,1360,1551
first,NaN,2018-01-03 12:45:52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,2021-03-21 22:00:23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
meat_cleaned.sample(2)

,order_num,order_placed,delivery_date,delivery_details,item,sku,qty,subtotal,order_total,shipping_and_handling,Grand Total,Online Payment,Amount Due,billing_address,shipping_address,shipment_method,payment_method
596,STHY159902,2019-11-11 09:53:00,"Nov 12, 2019",8:00am - 11:59am,[Silver Pomfret / Avoli (100g to 200g) - Whole...,[silver-pomfret-medium],[0.5 kg],[822.00],822.00,29.00,822.00,762.00,0.00,VAISHALI Hatkar #Bld.No 231 Room No 9044 B win...,VAISHALI Hatkar #Bld.No 231 Room No 9044 B win...,Home Delivery - Flat,Online Payment
550,STHB3621404,2020-08-29 16:55:50,"Aug 30, 2020",7:30am - 9:30am,[Premium Tender and Antibiotic-residue-free Ch...,[chicken-curry-cut],[1 Kg],[238.00],238.00,29.00,267.00,267.00,0.00,"Sathish Kumar.J Christina nilayam,2nd floor,17...","Sathish Kumar.J Christina nilayam,2nd floor,17...",Home Delivery - Flat,Online Payment


### Renaming columns

In [78]:
# renaming specific columns in place
meat_cleaned.rename({'order_num': 'order_ID', 'order_placed': 'order_datetime', 'delivery_details': 'delivery_time', 'subtotal': 'subtotal (Rs)', 'order_total': 'order_total (Rs)', 'shipping_and_handling': 'shipping_handlingfees (Rs)', 'Grand Total': 'grand_total (Rs)', 'Online Payment': 'paid_online (Rs)', 'Amount Due': 'amount_due (Rs)'}, axis=1, inplace=True) 

In [79]:
meat_cleaned.isna().sum()

order_ID                        0
order_datetime                  0
delivery_date                   0
delivery_time                   0
item                           57
sku                            57
qty                            57
subtotal (Rs)                  57
order_total (Rs)               57
shipping_handlingfees (Rs)     58
grand_total (Rs)               57
paid_online (Rs)              429
amount_due (Rs)                57
billing_address                 0
shipping_address                0
shipment_method                 0
payment_method                  0
dtype: int64

In [80]:
# exporting the cleaned DF to csv

meat_cleaned.to_csv('cleanedData_meat.csv', index=False, encoding="utf-8")

## 4. Report after cleaning meat data

**We were able to retrieve 1941 rows with 18 columns of data. Out of them, half of them don't have any null values while the other half mostly has around 60 null which isn't relatively significant. The Online Payment has about 25% null values.**

# Working on flip_oc_0 JSON File

## 1. Parsing flip

### Initialising and processing data

In [81]:
# loading in our second file

with open('flip_oc_0.json', encoding="utf-8") as f:
    data_flip = json.load(f)

In [82]:
# transforming the json data into Pandas dataframe

id_list = []
html_list = []
to_list = []
from_list = []
company_list = []
subject_list = []

for row in data_flip:
    id = row["_id"]
    htmll = row["_source"]["rawHtml"]
    to = row["_source"]["to"]
    fromm = row["_source"]["from"]
    company = row["_source"]["companyName"]
    subject = row["_source"]["subject"]
    
    id_list.append(id)
    html_list.append(htmll)
    to_list.append(to)
    from_list.append(fromm)
    company_list.append(company)
    subject_list.append(subject)
    

In [83]:
df_flip = pd.DataFrame(list(zip(id_list, html_list, to_list, from_list, company_list, subject_list)), columns = ['id', 'raw_html', 'to_email', 'from_email', 'company', 'email_subject'])

In [84]:
df_flip.sample(4)

,id,raw_html,to_email,from_email,company,email_subject
713,hZZqjXgBPctInV7JF3UG,"<html xmlns=""http://www.w3.org/1999/a"" xmlns:v...",sureshwaran2681994@gmail.com,no-reply@nct.flipkart.com,flipkart,Your Order for RUNEECH CABLEPROTECTOR... has b...
1906,ZmjrhngBPctInV7Jq_WR,"<html xmlns=""http://www.w3.org/1999/a"" xmlns:v...",nilanjan007.laya@gmail.com,no-reply@nct.flipkart.com,flipkart,Your Order for Hawkins Contura Black ... has b...
914,12PEhngBPctInV7JC_YW,"<html xmlns=""http://www.w3.org/1999/a"" xmlns:v...",rajeevs446@gmail.com,no-reply@nct.flipkart.com,flipkart,Your Order for Salcon Electronics BT ... has b...
1891,C2jqhngBPctInV7J4dy3,"<html xmlns=""http://www.w3.org/1999/a"" xmlns:v...",gangaramattri786@gmail.com,no-reply@nct.flipkart.com,flipkart,Your Order for TheFashionPlus Solid M... has b...


### Doing EDA on our DF

In [85]:
df_flip.describe()

,id,raw_html,to_email,from_email,company,email_subject
count,2000,2000,2000,2000,2000,2000
unique,2000,1845,1331,4,1,1416
top,kGPDhngBPctInV7J8fPb,"<html xmlns=""http://www.w3.org/1999/a"" xmlns:v...",rajababu0090@gmail.com,no-reply@nct.flipkart.com,flipkart,Your Order for Saffola Oats has been successfu...
freq,1,5,16,1100,2000,24


In [86]:
df_flip.to_email.value_counts()

rajababu0090@gmail.com          16
rakeshkumar111rky@gmail.com     11
kamlesh08087@gmail.com          11
pratikdebbarma27@gmail.com       9
riftikar7@gmail.com              9
                                ..
vishnugkannath@gmail.com         1
sutradharsenjuti@gmail.com       1
purusottambehera24@gmail.com     1
pratikmishra4567@gmail.com       1
rafiqs460@gmail.com              1
Name: to_email, Length: 1331, dtype: int64

In [87]:
df_flip.from_email.value_counts()

no-reply@nct.flipkart.com    1100
no-reply@flipkart.com         502
no-reply@ncp.flipkart.com     199
no-reply@ncb.flipkart.com     199
Name: from_email, dtype: int64

In [88]:
# checking to see if there's any structure differences based on email sender

print(df_flip[df_flip["from_email"] == 'no-reply@nct.flipkart.com']["id"].tail(1))
print(df_flip[df_flip["from_email"] == 'no-reply@flipkart.com']["id"].head(1))
print(df_flip[df_flip["from_email"] == 'no-reply@ncp.flipkart.com']["id"].head(1))
print(df_flip[df_flip["from_email"] == 'no-reply@ncb.flipkart.com']["id"].head(1))
print(df_flip[df_flip["to_email"] == 'rajababu0090@gmail.com']["id"].head(1))

1996    BGv9hngBPctInV7JLTGQ
Name: id, dtype: object
0    kGPDhngBPctInV7J8fPb
Name: id, dtype: object
1    p2XQhngBPctInV7J-ZwL
Name: id, dtype: object
30    qHRJh3gBPctInV7Jt6R1
Name: id, dtype: object
1299    XmXNhngBPctInV7JgiaO
Name: id, dtype: object


### Parsing on sample HTML

In [89]:
# parsing one sample html

one_flip = df_flip.iloc[2]["raw_html"]
soup = BeautifulSoup(one_flip, 'html.parser')

In [90]:
# parsing one sample html

onee_flip = df_flip.iloc[1996]["raw_html"]
soupp = BeautifulSoup(onee_flip, 'html.parser')
soupp.get_text()

"\n\n\n\n\n\n\nFlipkart.com\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nOrder Placed\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHi  Customer,  Your order has been successfully placed.\n\n\n\n\n\n\nOrder placed on Feb 27, 2021Order ID OD121125680663755000\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nWe are committed to serving you with utmost regard for your safety. Know more about our precautionary measures here. Please note, the delivery date of your order may change based on the government's zonal advisory in your area. \n\n\n\n\n\n\n\n\n\n Delivery by Sat, Mar 06, 2021  Amount PayableRs. 458 Manage Your Order \n\n\n\n\n\n\n\n\nDelivery Address\n Bittu Sonowal   New arts building foreign languages dept  Gauhati University  Guwahati, Assam, 781014 \nSMS updates sent to9101494537\n\n\n\n\n\n\n You will receive the next update when the item in your order is packed/shipped by the seller. \n\n\n\n\n\n\n\n\nYour Rewards with this order\n\n\n\n\n\n\n\n\n\n\n\n 16 SuperCoins On The Way \n\n\n Will be credited afte

In [91]:
# getting just the text corpus from our rawHtml

soup.get_text()

'             Flipkart.com                       Order Placed       \n            \n      Hi                      Harshal Mevada,  Your order has been successfully placed.         Ordered \r\nplaced on Feb 03, 2021  Order ID OD120920728474237000              The details of the Gift Card, including the Gift Card ID, PIN and Expiry, have been sent to the respective \r\nrecipients.    View Order       \n       Please find below, the summary of your order    Product details Quantity Amount Sub total           E-Gift Voucher  (To: harshalmevada@gmail.com)   \n    1 �. 25 �. 25                   Total    �. 25 \n               \n   Thank you for shopping with Flipkart!  Got Questions? Please get in touch with our 24x7 Customer Care    \n                                           This email was sent from a notification-only address that cannot accept incoming email. Please do not reply to this message.       \n                   \xa0  '

---

We see that there's a couple of details we can extract. We have:
- the person name
- order placed date 
- order ID
- product detail
- quantity
- amount
- subtotal


In [92]:
# formatting and removing spaces from our text corpus

all_text = soup.get_text()
all_text = all_text.split('\n')
text = [t.strip() for t in all_text]
text = [" ".join(re.split("\s+", t, flags=re.UNICODE)) for t in text]
text = list(filter(None, text))
text = " ".join(text)

text

'Flipkart.com Order Placed Hi Harshal Mevada, Your order has been successfully placed. Ordered placed on Feb 03, 2021 Order ID OD120920728474237000 The details of the Gift Card, including the Gift Card ID, PIN and Expiry, have been sent to the respective recipients. View Order Please find below, the summary of your order Product details Quantity Amount Sub total E-Gift Voucher (To: harshalmevada@gmail.com) 1 �. 25 �. 25 Total �. 25 Thank you for shopping with Flipkart! Got Questions? Please get in touch with our 24x7 Customer Care This email was sent from a notification-only address that cannot accept incoming email. Please do not reply to this message.'

### Building the Parser for flip DF with Regex

In [93]:
records_flip = []

def extract_flip(flip_html):
    
    # extracting whole text corpus
    soup = BeautifulSoup(flip_html, 'html.parser')
    all_text = soup.get_text()
    all_text = all_text.split('\n')
    text = [t.strip() for t in all_text]
    text = [" ".join(re.split("\s+", t, flags=re.UNICODE)) for t in text]
    text = list(filter(None, text))
    text = " ".join(text)


    # defining patterns
    pat_personname = re.compile(r'hi\s(.+?),', re.IGNORECASE)
    pat_orderdate = re.compile(r'on\s(\w+\s\d+,\s\d{4})', re.IGNORECASE)
    pat_orderid = re.compile(r'order\sid\s(\w+)', re.IGNORECASE)
    pat_deliverydate = re.compile(r'delivery by\s(.+?\d{4})', re.IGNORECASE)
    pat_amountpaid = re.compile(r'amount.+? (\d+)', re.IGNORECASE)
    pat_deliveradr = re.compile(r'delivery address\s(.+?\d{6})', re.IGNORECASE)
    pat_phonenum = re.compile(r'sent to(\d{10})', re.IGNORECASE)
    pat_coinaward = re.compile(r'(\d)\ssupercoins', re.IGNORECASE)
    pat_itembought = re.compile(r'(?:is over|click here)\s([\w\s]+).+?(?:\d+\.\d)', re.IGNORECASE)
    pat_itembought2 = re.compile(r'plus members\s(.+)\s.+?\s(?:\d+\.\d)', re.IGNORECASE)
    pat_itembought3 = re.compile(r'(?:is over|click here).?\s([\w\s,-]+?)\s(?:Rs\.\s\d+\.\d)', re.IGNORECASE)
    pat_itembought4 = re.compile(r'sub\stotal\s(.*?)\s(?:\(to:)', re.IGNORECASE)
    pat_sellername = re.compile(r'seller:\s(.+?)\s(?:delivery|qty)', re.IGNORECASE)
    pat_deliverycharges = re.compile(r'delivery charges.+?(\d+).+?qty', re.IGNORECASE)
    pat_itemquantity = re.compile(r'qty.+?(\d+?)\s?\w+?', re.IGNORECASE)

    
    # searching for matches
    pname = pat_personname.search(text)
    pdate = pat_orderdate.search(text)
    oid = pat_orderid.search(text)
    del_date = pat_deliverydate.search(text)
    amt_paid = pat_amountpaid.search(text)
    del_adr = pat_deliveradr.search(text)
    phonenum = pat_phonenum.search(text)
    supercoin_award = pat_coinaward.search(text)
    item_bought = pat_itembought.search(text)
    item_bought2 = pat_itembought2.search(text)
    item_bought3 = pat_itembought3.search(text)
    item_bought4 = pat_itembought4.search(text)
    seller_name = pat_sellername.search(text)
    delivery_charges = pat_deliverycharges.search(text)
    item_quantity = pat_itemquantity.search(text)

    
    # validating
    if pname:
        pname = pname.group(1)
    else:
        pname = None
    
    if pdate:
        pdate = pdate.group(1)
    else:
        pdate = None
        
    if oid:
        oid = oid.group(1)
    else:
        oid = None
        
    if del_date:
        del_date = del_date.group(1)
    else:
        del_date = None
        
    if amt_paid:
        amt_paid = amt_paid.group(1)
    else:
        amt_paid = None
        
    if del_adr:
        del_adr = del_adr.group(1)
    else:
        del_adr = None
        
    if phonenum:
        phonenum = phonenum.group(1)
    else:
        phonenum = None
        
    if supercoin_award:
        supercoin_award = supercoin_award.group(1)
    else:
        supercoin_award = None
        
    if item_bought:
        item_bought = item_bought.group(1)
    elif item_bought2:
        item_bought = item_bought2.group(1)
    elif item_bought3:
        item_bought = item_bought3.group(1)
    elif item_bought4:
        item_bought = item_bought4.group(1)
    else:
        item_bought = None
        
    if seller_name:
        seller_name = seller_name.group(1)
    else:
        seller_name = None
    
    if delivery_charges:
        delivery_charges = delivery_charges.group(1)
    else:
        delivery_charges = None
        
    if item_quantity:
        item_quantity = item_quantity.group(1)
    else:
        item_quantity = None
    
    
    # appending all mined data to records_flip
    records_flip.append((text, pname, pdate, oid, del_date, amt_paid, del_adr, phonenum, supercoin_award, item_bought, seller_name, delivery_charges, item_quantity))

In [94]:
for x in df_flip["raw_html"]:
    extract_flip(x)

In [95]:
# making a pandas DF out of our generated list

data_flip = pd.DataFrame(records_flip, columns=["text", "pname", "pdate", "oid", "del_date", "amt_paid", "del_adr", "phonenum", "supercoin_award", "item_bought", "seller_name", "delivery_charges", "item_quantity"])
data_flip.sample(5)

,text,pname,pdate,oid,del_date,amt_paid,del_adr,phonenum,supercoin_award,item_bought,seller_name,delivery_charges,item_quantity
477,"Flipkart.com Order Placed Hi Abhilash Nair, Yo...",Abhilash Nair,"Feb 01, 2021",OD220905810882902000,"Mon, Feb 08, 2021",375,"Abhilash Nair Flat 20/B JM Medicine Corner, Su...",9088864775,2,Friendskart Personalized Photo and Text Cerami...,Friendskart.in,40,1
1083,"Flipkart.com Order Placed Hi Mithilesh kumar, ...",Mithilesh kumar,"Feb 04, 2021",OD120930835345002000,"Sun, Feb 07, 2021",977,Mithilesh WZ-167C Madipur Village near noorani...,7042227908,6,Woodland Monza Canvas Shoes For Men,Wizrob Fashion,40,1
958,"Flipkart.com Order Placed Hi Arjun PALASANIYA,...",Arjun PALASANIYA,"Feb 19, 2021",OD121057597738281000,"Sat, Feb 20, 2021",356,Arjun Palsaniya 75 jagdish palsaniya ki dhani ...,9610732413,2,Head & Shoulders Smooth & Silky Shampoo,SuperComNet,40,1
1599,"Flipkart.com Order Placed Hi PYDIMALLA Raju, Y...",PYDIMALLA Raju,"Feb 15, 2021",OD121022479560216000,"Sat, Feb 20, 2021",138,Pydimalla Ratnaraju 4-86 Durga amma vari stree...,9676556156,4,Futaba Adjustable Baby Shower Cap,YMBRYNE,40,1
1609,"Flipkart.com Order Placed Hi Abijith Krish, Yo...",Abijith Krish,"Feb 22, 2021",OD121086134165948000,"Mon, Mar 01, 2021",279,"EYSYS PHARMACEUTICAL PVT LTD Nr.SBI, 100 FT RO...",6238319491,8,DivineDesigns Medium Johan Cruyff Sticker ( Si...,DivineDesigns,45,1


In [96]:
data_flip.describe()

,text,pname,pdate,oid,del_date,amt_paid,del_adr,phonenum,supercoin_award,item_bought,seller_name,delivery_charges,item_quantity
count,2000,2000,2000,2000,1933,2000,1978,1978,1911,1720,1978,1857,1978
unique,1845,1243,59,1845,71,670,1418,1404,5,1196,929,53,6
top,"Flipkart.com Order Placed Hi Saurav Mukherjee,...",Flipkart Customer,"Feb 03, 2021",OD120905832808365000,"Wed, Feb 17, 2021",1,Sheikh Yashif Barahagoda infront of forest che...,7749028317,4,Saffola Oats,SuperComNet,40,1
freq,5,65,77,5,77,41,12,12,547,25,241,1408,1903


In [97]:
# dropping the text column, which was included for debugging purposes

data_flip.drop(data_flip.columns[[0]], axis = 1, inplace=True)

---

## 2. Report after parsing flip data

**Out of a total of 2000 mails, we were able to extract values in all our columns. However, we do see almost all columns missing 22 records which we will investigate in the Cleaning process. Thus, 100% of the data was parsed with our Regex with the information in the corpus.**

### Printing parsed flip data to CSV

In [98]:
# printing the parsed data of meat JSON

data_flip.to_csv('parsedData_flip.csv', index=False)

## 3. Cleaning the parsed flip data

In [99]:
data_flip.sample(5)

,pname,pdate,oid,del_date,amt_paid,del_adr,phonenum,supercoin_award,item_bought,seller_name,delivery_charges,item_quantity
1304,Shaik Yashif raja,"Feb 13, 2021",OD121003380572229000,"Fri, Feb 19, 2021",269,Sabina Parveen Basna infront of bsnl office Ad...,9753043622,8,Swiss Beauty Primer & Mattifying Powder Primer...,Gabru Raja,40,1
272,Biplab Sarkar 19053009518920N4,"Feb 06, 2021",OD220946029957532000,"Sat, Feb 13, 2021",286,"Biplab Sarkar 10/907 Malviya Nagar, Siddharth ...",9007468167,8,ADIDAS Slippers,Decor k,45,1
313,Roshan Lal,"Feb 13, 2021",OD221008420102258000,"Fri, Feb 19, 2021",644,Roshan Lal Saidpur Mishrauli Jaysinghpur Road ...,8628025521,4,None,VANDANA FASHION,60,1
930,Ashutosh Dalal,"Feb 11, 2021",OD120991929546760000,"Wed, Feb 17, 2021",359,Prity Maji LH 1 hostel Bidhan chandra krishi v...,8509755344,2,Zombom Casual Regular Sleeve Solid Women Blue Top,THE ZB,40,1
900,Sarthi Shah,"Mar 09, 2021",OD121215064853233000,"Thu, Mar 11, 2021",7999,"Sarthi Shah 75, Maniratnam - II, Vasna, Ahmeda...",9558314821,0,Fitbit Inspire,RetailNet,40,1


### Renaming columns with proper names

In [100]:
# renaming specific columns in place
data_flip.rename({'pname': 'person_name', 'pdate': 'purchase date', 'oid': 'order_ID', 'del_date': 'delivery_date', 'amt_paid': 'amount_paid (Rs)', 'del_adr': 'delivery_address', 'phonenum': 'phone_number', 'supercoin_award': 'supercoins_awarded', 'delivery_charges': 'delivery_charges (Rs)'}, axis=1, inplace=True) 

In [101]:
data_flip.columns

Index(['person_name', 'purchase date', 'order_ID', 'delivery_date',
       'amount_paid (Rs)', 'delivery_address', 'phone_number',
       'supercoins_awarded', 'item_bought', 'seller_name',
       'delivery_charges (Rs)', 'item_quantity'],
      dtype='object')

In [102]:
data_flip.sample(5)

,person_name,purchase date,order_ID,delivery_date,amount_paid (Rs),delivery_address,phone_number,supercoins_awarded,item_bought,seller_name,delivery_charges (Rs),item_quantity
154,Flipkart Customer,"Feb 10, 2021",OD120980183420866000,"Thu, Feb 18, 2021",199,Rahul Gogoi 165 Maruti Anamika motors Sibsagar...,9365803621,4,Aadi Slides,Dadwanis Footwear,40,1
367,Saurav Mukherjee,"Feb 01, 2021",OD120905832808365000,"Mon, Feb 08, 2021",1277,"Saurav Mukherjee 35HCP6, Amzam Tower Bharathi ...",8054233429,8,Herbalife Personalize Protein Powder Protein B...,BIG BRAND,50,1
377,bhaskar sharma,"Feb 25, 2021",OD221109511798038000,"Fri, Feb 26, 2021",689,Bhaskar Sharma Vista console electronics B 103...,9412195663,2,Signoraware Executive Glass 3 Containers Lunch...,RetailNet,70,1
1602,julius nadar,"Feb 02, 2021",OD220912366525820000,None,1,"Julius Nadar Room no 2, Indira building, sanka...",9594508744,None,None,SuperComNet,None,1
33,Ashutosh Agrawal,"Feb 14, 2021",OD121016704591867000,"Fri, Feb 19, 2021",298,"Ashutosh Agrawal Ambica Agro Industries, Club ...",8867355252,8,Slides,TATHAGAT ENTERPRISES,60,1


### Removing incorrect entries

In [103]:
data_flip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   person_name            2000 non-null   object
 1   purchase date          2000 non-null   object
 2   order_ID               2000 non-null   object
 3   delivery_date          1933 non-null   object
 4   amount_paid (Rs)       2000 non-null   object
 5   delivery_address       1978 non-null   object
 6   phone_number           1978 non-null   object
 7   supercoins_awarded     1911 non-null   object
 8   item_bought            1720 non-null   object
 9   seller_name            1978 non-null   object
 10  delivery_charges (Rs)  1857 non-null   object
 11  item_quantity          1978 non-null   object
dtypes: object(12)
memory usage: 187.6+ KB


After checking the CSV manually, we can see that there are certain errors in detecting the amount paid for Gift Vouchers and it doesn’t hold correct data. Also, most columns for gift vouchers are blank since their email structure is different and don’t contain much information so we can remove these rows. There are 22 such rows so it shouldn’t affect our future insights by a huge margin.

In [104]:
data_flip[data_flip["delivery_address"].isnull()]

,person_name,purchase date,order_ID,delivery_date,amount_paid (Rs),delivery_address,phone_number,supercoins_awarded,item_bought,seller_name,delivery_charges (Rs),item_quantity
0,Manashi Saha,"Feb 21, 2021",OD121076230805862000,None,1,None,None,None,E-Gift Voucher,None,None,None
1,Nigaran Gs,"Feb 08, 2021",OD120957862971179000,None,1,None,None,None,E-Gift Voucher,None,None,None
2,Harshal Mevada,"Feb 03, 2021",OD120920728474237000,None,1,None,None,None,E-Gift Voucher,None,None,None
3,S khabar basha,"Feb 13, 2021",OD121006736143853000,None,1,None,None,None,E-Gift Voucher,None,None,None
4,Manashi Saha,"Feb 21, 2021",OD121076230805862000,None,1,None,None,None,E-Gift Voucher,None,None,None
5,Nigaran Gs,"Feb 08, 2021",OD120957862971179000,None,1,None,None,None,E-Gift Voucher,None,None,None
6,Harshal Mevada,"Feb 03, 2021",OD120920728474237000,None,1,None,None,None,E-Gift Voucher,None,None,None
7,Naveen Saini,"Mar 01, 2021",OD221143168203016000,None,1,None,None,None,E-Gift Voucher,None,None,None
8,Naveen Saini,"Mar 01, 2021",OD221143259832272000,None,1,None,None,None,E-Gift Voucher,None,None,None
9,Naveen Saini,"Mar 01, 2021",OD221143293851722000,None,1,None,None,None,E-Gift Voucher,None,None,None


In [105]:
dfflip_cleaned = data_flip[data_flip["delivery_address"].notnull()]
dfflip_cleaned

,person_name,purchase date,order_ID,delivery_date,amount_paid (Rs),delivery_address,phone_number,supercoins_awarded,item_bought,seller_name,delivery_charges (Rs),item_quantity
22,Pankaj Boricha,"Feb 11, 2021",OD220988471815965000,"Tue, Feb 23, 2021",2539,"Pankaj ""PANKAJ BLECKENING"" Navrang para street...",9825881909,0,Minecraft,GAMERSGALAXY,40,1
23,Suraj Biswakarma,"Feb 13, 2021",OD121005793535846000,"Mon, Feb 15, 2021",199,"Suraj Lohar Sri Ramakrishna Hospital No: 395, ...",9101425807,4,Slides,Dadwanis Footwear,40,1
24,Ahmad Abbas,"Feb 16, 2021",OD121028100603673000,"Sat, Feb 20, 2021",299,"Ahmad Abbas 22 - A/50 Rahmat Nagar, Kareli Pra...",7897177756,8,Slippers,ParagonFootwear,40,1
25,Customer,"Feb 13, 2021",OD121005790592542000,"Thu, Feb 18, 2021",539,"Dhiraj Lobo 207,mohan bhagat chawl, phanspada,...",9888200884,0,Slides,ParagonFootwear,40,1
26,George Thomas,"Feb 07, 2021",OD120954896578702000,"Tue, Feb 16, 2021",324,George Thomas Mammootil Cherukattu Kuttapuzha ...,8129238533,2,Freakonomics,Amazingbook,62,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Usman Choudhary,"Feb 05, 2021",OD120937253622329000,"Sun, Feb 07, 2021",711,"Usman Chaudhary Mnc colony , jadhav wadi Jadha...",9168445249,8,None,RetailNet,40,1
1996,Customer,"Feb 27, 2021",OD121125680663755000,"Sat, Mar 06, 2021",458,Bittu Sonowal New arts building foreign langua...,9101494537,6,Highlander Men Striped Casual Blue Shirt,Wizrob Fashion,40,1
1997,Masthan Bassha,"Feb 25, 2021",OD121105528432726000,"Sat, Mar 06, 2021",604,Masthanbassha 124/20 SriDevi illam Manali new ...,8778463677,4,None,TownWalker,55,1
1998,Swaraj Roy,"Feb 24, 2021",OD221098968916060000,"Mon, Mar 01, 2021",489,Swaraj 12 Mansurchak Subdistrict Begusarai Dis...,7352194715,8,Lovisa Fashion Embroidered Semi Stitched Lehen...,LovisaFashion,40,1


In [106]:
dfflip_cleaned.describe()

,person_name,purchase date,order_ID,delivery_date,amount_paid (Rs),delivery_address,phone_number,supercoins_awarded,item_bought,seller_name,delivery_charges (Rs),item_quantity
count,1978,1978,1978,1933,1978,1978,1978,1911,1698,1978,1857,1978
unique,1227,59,1826,71,670,1418,1404,5,1195,929,53,6
top,Customer,"Feb 03, 2021",OD120905832808365000,"Wed, Feb 17, 2021",299,Sheikh Yashif Barahagoda infront of forest che...,7749028317,4,Saffola Oats,SuperComNet,40,1
freq,65,75,5,77,35,12,12,547,25,241,1408,1903


In [107]:
dfflip_cleaned.sample(5)

,person_name,purchase date,order_ID,delivery_date,amount_paid (Rs),delivery_address,phone_number,supercoins_awarded,item_bought,seller_name,delivery_charges (Rs),item_quantity
1176,Ritesh Jawarkar,"Feb 14, 2021",OD121017493912078000,"Sat, Feb 20, 2021",121,"Ritesh Jawarkar kolar type 3, 40 dk honey home...",7509339399,4,RUNEECH Back Tempered Glass for Realme U1,AVRENTERPRISES,40,1
701,Sk Nizam uddin,"Mar 28, 2021",OD221377890271148000,"Tue, Apr 06, 2021",175,"Sk Nizamuddin 103A/69, Banamali ghosal lane be...",8981706983,4,"Naturalcreations Tail Light, Brake Light, Park...",Archika,55,1
393,Upendra Singh,"Feb 04, 2021",OD120927490846412000,"Wed, Feb 10, 2021",1549,Sonu Kumar Arvind kumar singh(Advocate )-house...,9065048704,0,Kechaoda K60,ssmobiles,40,1
1905,PAVAN CHAUHAN,"Feb 08, 2021",OD120961239313703000,"Thu, Feb 11, 2021",399,"PAVAN CHAUHAN 13 Yashovrund society Meghraj, G...",9723696113,2,SIDWA Women Nighty,RAJMAPRODUCTS,40,1
1632,Rishabh Jhamb,"Feb 19, 2021",OD121056751174655000,"Tue, Feb 23, 2021",249,Rishabh Jhamb 1066 sector 67 mohali punjab 160062,8557946448,8,Manogyam mano weighing Weighing Scale,Royal_Retail,40,1


In [108]:
dfflip_cleaned.isna().sum()

person_name                0
purchase date              0
order_ID                   0
delivery_date             45
amount_paid (Rs)           0
delivery_address           0
phone_number               0
supercoins_awarded        67
item_bought              280
seller_name                0
delivery_charges (Rs)    121
item_quantity              0
dtype: int64

### Printing cleaned flip data to CSV

In [109]:
# printing the cleaned data of flip JSON

dfflip_cleaned.to_csv('cleanedData_flip.csv', index=False)

---

## 4. Report after cleaning flip data

**We were able to retain 1978 rows out of 2000 rows, with 12 columns of data, coming to about 99% retrieval of the entire dataset. 75% of the columns have only non-null values. The column with maximum null values is item_bought, from which we retrieved 86% of the values completely. This can be due to more variations in the formatting of the text present in the rawHtml.**